# Preprocess MEOP dataset

Dataset obtained from: https://www.meop.net/download-the-data.html

Date accessed: 1st September, 2023

Dataset version: 26th November, 2021

In [1]:
import xarray as xr
import pandas as pd
import polars as pl
import numpy as np
import gsw
import glob
import os

### List all the files with the full path

In [2]:
!find /scratch/an1n23/MEOP-CTD_2021-11-26/ -type f -name '*.nc'

/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ct132-331-16_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft23-C902-18_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft23-C900-18_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ct120-266BAT-15_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft18-F4-12_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ct139-331BAT-16_traj.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft22-882-18_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ct139-F932BAT-13_traj.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft22-881-18_traj.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft22-686-18_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft22-879-18_traj.nc
/scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DATA_FULL_RES/ft22-879-18_fr1_prof.nc
/scratch/an1n23/MEOP-CTD_2021

### Now write all the filepaths into a txt file

In [3]:
!rm ./filelistMEOP.txt

In [4]:
!find /scratch/an1n23/MEOP-CTD_2021-11-26/ -type f -name '*.nc' > filelistMEOP.txt

In [2]:
filelistMEOP = pd.read_csv("./filelistMEOP.txt", header=None)

In [3]:
filelistMEOP[0]

0       /scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DAT...
1       /scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DAT...
2       /scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DAT...
3       /scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DAT...
4       /scratch/an1n23/MEOP-CTD_2021-11-26/FRANCE/DAT...
                              ...                        
1797    /scratch/an1n23/MEOP-CTD_2021-11-26/UK/DATA/ct...
1798    /scratch/an1n23/MEOP-CTD_2021-11-26/UK/DATA/ct...
1799    /scratch/an1n23/MEOP-CTD_2021-11-26/UK/DATA/ct...
1800    /scratch/an1n23/MEOP-CTD_2021-11-26/UK/DATA/ct...
1801    /scratch/an1n23/MEOP-CTD_2021-11-26/UK/DATA/ct...
Name: 0, Length: 1802, dtype: object

In [4]:
def create_merged_df(filenames):
    dfm = []
    for i in range((len(filenames))):
        print(i)
        with xr.open_dataset(filenames[0][i]) as arr_data:
            try:
                #print("Entering 1st try block")
                nlev = len(arr_data['PRES_ADJUSTED'][0])
                nprof = len(arr_data['PRES_ADJUSTED'])
                if(i == 0):
                    NPROF = np.arange(nprof)
                    NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])
                else:
                    last_end = NPROF.flatten()[-1] + 1
                    NPROF = np.arange(last_end, last_end+nprof )
                    NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])

                ind = np.arange(nprof*nlev)
                lat = np.array([[arr_data["LATITUDE"][j].values]* nlev for j in range(nprof) ])
                lon = np.array([[arr_data["LONGITUDE"][j].values]* nlev for j in range(nprof) ])
                posqc = np.array([[arr_data["POSITION_QC"][j].str.decode("utf-8").astype("int").values]* nlev for j in range(nprof) ])
                juld = np.array([[pd.to_datetime(arr_data["JULD"][j].values.item().isoformat())]* nlev for j in range(nprof) ])
                    

                if(i == 0):    
                    df = {'PLATFORM_NUMBER': pd.Series([int(arr_data["PLATFORM_NUMBER"][0].str.decode("utf-8").values)]*len(ind), 
                                                        index=ind), 
                          'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                          'TEMP_ADJUSTED': pd.Series(arr_data["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                          'PSAL_ADJUSTED': pd.Series(arr_data["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                          'PRES_ADJUSTED': pd.Series(arr_data["PRES_ADJUSTED"].values.flatten(), index=ind),  
                          'PRES_ADJUSTED_QC': pd.Series(arr_data["PRES_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind), 
                          'PRES_ADJUSTED_ERROR':pd.Series(arr_data["PRES_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                          'TEMP_ADJUSTED_QC': pd.Series(arr_data["TEMP_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind), 
                          'TEMP_ADJUSTED_ERROR': pd.Series(arr_data["TEMP_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                          'PSAL_ADJUSTED_QC': pd.Series(arr_data["PSAL_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind),  
                          'PSAL_ADJUSTED_ERROR':pd.Series(arr_data["PSAL_ADJUSTED_ERROR"].values.flatten(), index=ind),  
                          'JULD': pd.Series(juld.flatten(), index=ind),  
                          'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                          'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                          'POSITION_QC': pd.Series(posqc.flatten(), index=ind) }
                    dfm.append(pd.DataFrame(df))
                else:
                    try:
                        #print("Entering 2nd try block")
                        df_i = {'PLATFORM_NUMBER': pd.Series([int(arr_data["PLATFORM_NUMBER"][0].str.decode("utf-8").values)]*len(ind), 
                                                             index=ind), 
                          'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                          'TEMP_ADJUSTED': pd.Series(arr_data["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                          'PSAL_ADJUSTED': pd.Series(arr_data["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                          'PRES_ADJUSTED': pd.Series(arr_data["PRES_ADJUSTED"].values.flatten(), index=ind),  
                          'PRES_ADJUSTED_QC': pd.Series(arr_data["PRES_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind), 
                          'PRES_ADJUSTED_ERROR':pd.Series(arr_data["PRES_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                          'TEMP_ADJUSTED_QC': pd.Series(arr_data["TEMP_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind), 
                          'TEMP_ADJUSTED_ERROR': pd.Series(arr_data["TEMP_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                          'PSAL_ADJUSTED_QC': pd.Series(arr_data["PSAL_ADJUSTED_QC"].str.decode("utf-8").astype("int").values.flatten(), 
                                                        index=ind),  
                          'PSAL_ADJUSTED_ERROR':pd.Series(arr_data["PSAL_ADJUSTED_ERROR"].values.flatten(), index=ind),  
                          'JULD': pd.Series(juld.flatten(), index=ind),  
                          'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                          'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                          'POSITION_QC': pd.Series(posqc.flatten(), index=ind)  }

                        df_i = pd.DataFrame(df_i)
                        dfm.append(df_i)
                    except:
                        #print("Inner except block triggered")
                        pass

            except:
                #print("Outer except block triggered")
                pass

    return dfm


In [ ]:
dfm = create_merged_df(filelistMEOP)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


In [ ]:
df_merged = pd.concat(dfm, ignore_index=True)

### Save the merged dataframe to disk

In [ ]:
df_merged.to_csv("/home/an1n23/datasets/merged_MEOP_26112021.csv")

In [ ]:
print("completed merged MEOP disk write")

In [ ]:
!echo "completed meop disk write 26092023" > arbit_meop.txt

In [15]:
!ls /home/an1n23/datasets/MEOP_26112021/

MEOP-CTD_2021-11-26_ALL.zip  home  merged_MEOP_26112021.csv


### Clean up the files that are no longer required

Delete the original dataset

In [16]:
!rm -r /home/an1n23/datasets/MEOP_26112021/MEOP-CTD_2021-11-26_ALL.zip /home/an1n23/datasets/MEOP_26112021/home

In [17]:
df_merged.columns

Index(['DEPLOYMENT_CODE', 'SMRU_PLATFORM_CODE', 'CYCLE_NUMBER', 'JULD',
       'LATITUDE', 'LONGITUDE', 'N_TEMP', 'N_PSAL', 'N_CHLA', 'year', 'month',
       'day', 'MASK', 'N_PROF_TEMP', 'N_PROF_PSAL', 'N_PROF_CHLA', 'N_TAGS',
       'PI_CODE', 'PROCESS', 'PUBLIC', 'COUNTRY', 'FIRST_VERSION',
       'LAST_VERSION', 'T1', 'T2', 'S1', 'S2', 'remove', 'Sremove', 'comments',
       'variable_offset', 'instr_id', 'period', 'continuous', 'prefix'],
      dtype='object')

In [19]:
df_merged["comments"]

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1389524    NaN
1389525    NaN
1389526    NaN
1389527    NaN
1389528    NaN
Name: comments, Length: 1389529, dtype: object

In [21]:
df[0].columns

Index(['DEPLOYMENT_CODE', 'SMRU_PLATFORM_CODE', 'CYCLE_NUMBER', 'JULD',
       'LATITUDE', 'LONGITUDE', 'N_TEMP', 'N_PSAL', 'N_CHLA', 'year', 'month',
       'day', 'MASK'],
      dtype='object')

In [30]:
df[0].iloc[:, 6]

0       4.0
1       NaN
2       4.0
3       4.0
4       4.0
       ... 
209    23.0
210     NaN
211     NaN
212     NaN
213    23.0
Name: N_TEMP, Length: 214, dtype: float64